通常我们采用的是多项式 Hash 的方法，即 $f(s) = \sum s[i] \times b^i \pmod M$ ,
实际上应该是？ $f(s) = \sum s[i] \times b^{n-i} \pmod M => f_i(s) = f_{i-1}(s) \times b + s[i] \pmod M $ 。

这里面的 $b$ 和 $M$ 需要选取得足够合适才行，以使得 Hash 函数的值分布尽量均匀。

如果 $b$ 和 $M$ 互质，在输入随机的情况下，这个 Hash 函数在 $[0,M)$ 上每个值概率相等，此时单次比较的错误率为 $\frac 1 M$ 。所以，哈希的模数一般会选用大质数。

In [43]:

M=212370440130137957
B=233

def get_hash(s):
    res=0
    for c in s:
        #公式优化后的表达，简单全部求和之后，可能超过数据表达范围 
        res=int((res*B+ord(c))%M) 
    return res

M2=1e9+13
def get_hash2(s):
    res=0
    res2=0
    for c in s:
        #公式优化后的表达，简单全部求和之后，可能超过数据表达范围 
        res=(res*B+ord(c))%M 
        res2=(res2*B+ord(c))%M2 

    return res*res2

def cmp(s1,s2): 
    return get_hash(s1)==get_hash(s2)

def cmp2(s1,s2): 
    return get_hash2(s1)==get_hash2(s2)

cmp("aaab","aaab")
cmp2("aaab","aaabc")

False

### 多次询问子串哈希

单次计算一个字符串的哈希值复杂度是 $O(n)$ ，其中 $n$ 为串长，与暴力匹配没有区别，如果需要多次询问一个字符串的子串的哈希值，每次重新计算效率非常低下。

一般采取的方法是对整个字符串先预处理出每个前缀的哈希值，将哈希值看成一个 $b$ 进制的数对 $M$ 取模的结果，这样的话每次就能快速求出子串的哈希了：

令 $f_i(s)$ 表示 $f(s[1..i])$ ，那么 $f(s[l..r])=\frac{f_r(s)-f_{l-1}(s)}{b^{l-1}}$ ，其中 $\frac{1}{b^{l-1}}$ 也可以预处理出来，用 [乘法逆元](../math/inverse.md) 或者是在比较哈希值时等式两边同时乘上 $b$ 的若干次方化为整式均可。

这样的话，就可以在 $O(n)$ 的预处理后每次 $O(1)$ 地计算子串的哈希值了。

必备的入门操作，因为OI中用到的hash一般都是进制哈希，因为它有一些极其方便的性质，比如说，是具有和前缀和差不多的性质的。

$f(s[l..r])=f_r(s)-f_{l-1}(s)*b^{r-l-1} \pmod M$


In [59]:
M=212370440130137957
B=233
res=[0]

def get_hash_list(s):
    global res
    if len(res)>1:
        res=[0]
    for c in s:
        #公式优化后的表达，简单全部求和之后，可能超过数据表达范围 
        res.append(int((res[-1]*B+ord(c))%M))
    return

def get_sub_hash(i,j):
    if len(res)==1:
        print("gen hash list first")
    t=int((res[j]-res[i-1])/((B**(i-1))%M))
    t2=int((res[j]-res[i-1]*B**(j-i+1))%M)
    return t2


get_hash_list("abcdabc")
print(res)

print(get_sub_hash(2,3),get_sub_hash(6,7))
print(get_sub_hash(1,3),get_sub_hash(5,7))
print(get_hash("bc"))


[0, 97, 22699, 5288966, 1232329178, 287132698571, 66901918767141, 15588147072743952]
22933 22933
5288966 5288966
22933


现在有一个字符串S ，每次询问它的一个子串删除其中一个字符后的hash值（删除的字符时给定的）

要求必须 O(1) 回答询问

In [65]:
def del_sub_hash(x,l,r):
    if len(res)==1:
        print("gen hash list first")
    lh=get_sub_hash(l,x-1)
    rh=get_sub_hash(x+1,r)
    return (lh*B**(r-x)+rh)%M
print(del_sub_hash(3,2,5),get_hash("bda"))


5343719 5343719


In [67]:
class Solution:
    def longestPalindrome(self,s):
        hash_list=[0]*256
        res = 0
        for  c in s:
            hash_list[ord(c)]+=1
        for i in range(256):
            if hash_list[i] % 2 == 0: 
                res += hash_list[i]
                hash_list[i] = 0
            
            while hash_list[i] > 2: 
                res += hash_list[i] // 2 * 2
                hash_list[i] %= 2
            
        for i in range(256):
            if hash_list[i] > 0:
                res+=1
                break
        return res

Solution().longestPalindrome("aabcddcbefg")

9

In [ ]:
Each palindrome can be always created from the other palindromes, if a single character is also a palindrome. For example, the string "malayalam" can be created by some ways:

* malayalam = m + ala + y + ala + m
* malayalam = m + a + l + aya + l + a + m

We want to take the value of function NumPal(s) which is the number of different palindromes that can be created using the string S by the above method. If the same palindrome occurs more than once then all of them should be counted separately.

输入格式
The string S.

输出格式
The value of function NumPal(s).

示例：输入 "malayalam" 输出 15

In [77]:
#利用Hash求回文子串数量
base=13131
s="malayalam"
n=len(s)
h1=[0]*(n+2)
h2=[0]*(n+2)
p=[1]*(n+1)

def hash1(l,r):
    return int(h1[r]-h1[l-1]*p[r-l+1])

def hash2(l,r):
    return int(h2[l]-h2[r+1]*p[r-l+1])

def query_odd(x):
    l,r=1,min(x,n-x)
    while l<=r:
        mid=(l+r)>>1
        if hash1(x-mid,x+mid)==hash2(x-mid,x+mid):
            l=mid+1
        else:
            r=mid-1
    return r

def query_even(x):
    l,r=1,min(x,n-x)
    # l和r实际上是x坐标左右拓展范围
    while l<=r:
        mid=(l+r)>>1
        if hash1(x-mid+1,x+mid)==hash2(x-mid+1,x+mid):
            l=mid+1
        else:
            r=mid-1 
    return r  

for i in range(1,n+1):
    h1[i]=h1[i-1]*base+ord(s[i-1])
    p[i]=p[i-1]*base

for i in range(n,0,-1):
    h2[i]=h2[i+1]*base+ord(s[i-1])

ans=0
for i in range(1,n+1):
    x1=query_even(i)
    x2=query_odd(i)
    # print(i,x1,x2)
    ans+=x1+x2
print(ans+n)
        

15


In [88]:
#hash代替KMP算法
#给出两个字符串 S1 和 S2，其中 S2 为 S1 的子串，求 S2 在 S1 中出现多少次/出现的位置。

def substring(s1,s2):
    n=len(s1)
    m=len(s2)
    base=12312

    p=[1]*(n+1)
    h=[0]*(n+1)

    for i in range(1,n+1):
        p[i]=p[i-1]*base
        h[i]=h[i-1]*base+ord(s1[i-1])
    h2=0
    for i in range(m):
        h2=h2*base+ord(s2[i])

    res=[]   
    for i in range(n-m+1):
        hs=h[i+m]-h[i]*p[m]
        if hs==h2:
            res.append(i+1)
    return res

substring("aaccbbccbdds","ccb")

[3, 7]